# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **sports or athletics** (see below) for the region of **Ann Arbor, Michigan, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Ann Arbor, Michigan, United States** to Ann Arbor, USA. In that case at least one source file must be about **Ann Arbor, Michigan, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Ann Arbor, Michigan, United States** and **sports or athletics**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **sports or athletics**?  For this category we are interested in sporting events or athletics broadly, please feel free to creatively interpret the category when building your research question!

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [3]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from scipy.stats import norm
import matplotlib.mlab as mlab

#Import and clean datasets
def get_csrankings():
    csrankings = pd.read_excel('csranking_universities.xlsx')
    csrankings = csrankings.dropna()
    csrankings['Institution'] = csrankings['Institution'].apply(lambda x: re.split('►', x)[1].strip())
#     print(csrankings)
    return csrankings

def get_esports_universities():
    esports_universities = pd.read_excel('esports_universities.xlsx')
#     esports_universities = pd.Series(esports_universities)
    esports_universities = esports_universities.reset_index(drop=True)
    esports_universities = esports_universities['School']
#     esports_universities = pd.concat([pd.Series(['School']), esports_universities])
    esports_universities = esports_universities[3:]
#     esports_universities = esports_universities.values.reshape(365, 4, columns=['School', 'City and State', 'Region', 'League'])
    esports_universities = pd.DataFrame(np.reshape(esports_universities,(365,4)), columns=['School', 'City and State', 'Region', 'League'])
    esports_universities['City'] = esports_universities['City and State']
    esports_universities['City'] = esports_universities['City'].apply(lambda x: x.split(',')[0].strip())
    esports_universities['State'] = esports_universities['City and State']
    esports_universities['State'] = esports_universities['State'].apply(lambda x: x.split(',')[1].strip())
#     print(esports_universities)
    return esports_universities

def join_datasets_on_universities(inclusive=False):
    csrankings = get_csrankings()
    esports_universities = get_esports_universities()
    if inclusive:
        df = pd.merge(csrankings, esports_universities, how='left', right_on='School', left_on='Institution')
    else:
        df = pd.merge(csrankings, esports_universities, how='inner', right_on='School', left_on='Institution')
    return df

def chart_states_with_both():
    df = join_datasets_on_universities()
    df_incl = join_datasets_on_universities(inclusive=True)
    state_esports_counts = df.groupby('State').size().reset_index(name='count').sort_values(['count', 'State'], ascending=False)
#     print(state_esports_counts)
    
    
    #bar plot for states and how many esports teams they have
    plt.figure(figsize=(30,10))
    fig = plt.figure()
    f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(24,12),sharey=False, gridspec_kw={'width_ratios': [1, 2, 1]})
    f.suptitle('Top Ranked CS Program Institutions that have Esports teams', size=20)
    ax1.set_title('Number of Institutions per State')
    ax1.barh(range(len(list(state_esports_counts['State']))), state_esports_counts['count'], )
    plt.sca(ax1)
    plt.yticks(range(len(list(state_esports_counts['State']))), list(state_esports_counts['State']))
    
    #line plot for mean csrank of Institutions in state and how many esports teams they have
#     state_esports_counts['avgRank'] 
    df['#'] = df['#'].apply(lambda x: int(x))
    df_incl['#'] = df_incl['#'].apply(lambda x: int(x))
    df_incl = df_incl.sort_values('#')
    df2 = df
    df_incl = df_incl.reset_index()
    df2 = df2.reset_index(drop=True).groupby('State').mean()
    state_esports_ranks = pd.merge(state_esports_counts, df2, how='inner', left_on='State', right_index=True)
    state_esports_ranks = state_esports_ranks.sort_values(['count', 'State'], ascending=True)
    
    ax2.set_title('Average State CS Program Rank based on Institution Rank')
    ax2.plot(range(len(state_esports_ranks['count'])), state_esports_ranks['#'])
    plt.sca(ax2)
    plt.ylabel('Average CS Program Rank (Lower is Better)', rotation=90)
    plt.xticks(range(len(state_esports_ranks['count'])), list(state_esports_ranks['State']), rotation=75)
    
    state_esports_ranks = state_esports_ranks.reset_index()
    state_esports_ranks = state_esports_ranks.reset_index()
    state_esports_ranks['States Ordered by Number of Institutions that have both Esports teams and top ranked CS programs'] = state_esports_ranks['level_0']
    state_esports_ranks['Average CS Program Rank (Lower is Better)'] = state_esports_ranks['#']
    
    sns.regplot(x='States Ordered by Number of Institutions that have both Esports teams and top ranked CS programs', y='Average CS Program Rank (Lower is Better)' ,data=state_esports_ranks, fit_reg=True) 
    
    
    df_incl = df_incl[df_incl['School'].isin(df['School'].tolist())]
#     df_incl = df_incl[df_incl['index'] > 100]
    ax3.set_title('Probability of Esports programs within CS Rank Bins')
    n, bins, patches = ax3.hist(df_incl['index'], 15, normed=1)
#     hist(X[X$colA =="London" & X$colB == 10,]$colE)
#     (mu, sigma) = norm.fit(df_incl['index'].tolist())
    mu = df_incl['index'].mean()
    sigma = df_incl['index'].std()
    
    y = mlab.normpdf(bins, mu, sigma)
    print(y)
#     ax3.plot(bins, y, 'r--')
    plt.sca(ax3)
    plt.xlabel('Bins of Rank')
    plt.ylabel('Probability of Esports program')
    
    plt.show()
    
chart_states_with_both()

ValueError: Length of passed values is 365, index implies 1460.